In [1]:
import sys
sys.executable

'/Users/jiyeon/anaconda3/envs/datascience/bin/python'

Data Manipulation


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

train = pd.read_csv("../../heart_train.csv", index_col = 0)
test = pd.read_csv("../../heart_test.csv", index_col = 0)

def mapping(dataframe):
    dataframe["Sex"] = dataframe["Sex"].map({"M": 0, "F": 1})
    dataframe["ChestPainType"] = dataframe["ChestPainType"].map({"TA": 0, "ATA": 1, "NAP": 2, "ASY": 3})
    dataframe["RestingECG"] = dataframe["RestingECG"].map({"Normal": 0, "ST": 1, "LVH": 2})
    dataframe["ExerciseAngina"] = dataframe["ExerciseAngina"].map({"N": 0, "Y": 1})
    dataframe["ST_Slope"] = dataframe["ST_Slope"].map({"Up": 1, "Flat": 0, "Down": -1})

mapping(train)
mapping(test)

print((train).corr()["HeartDisease"].abs().sort_values(ascending = False))
# "RestingECG" has 0.08 correlation while "id" has 0.06

y = train["HeartDisease"]

train["RestingECG"].value_counts()
test["RestingECG"].value_counts()

HeartDisease      1.000000
ST_Slope          0.550568
ExerciseAngina    0.502934
ChestPainType     0.460001
Oldpeak           0.401702
MaxHR             0.390190
Sex               0.331767
FastingBS         0.307575
Age               0.299571
Cholesterol       0.246773
RestingBP         0.139909
RestingECG        0.080356
id                0.061128
Name: HeartDisease, dtype: float64


RestingECG
0    163
2     58
1     54
Name: count, dtype: int64

Random Forst Set up

In [3]:
features_columns = train.columns.values.tolist()
features_columns.remove("id")
features_columns.remove("HeartDisease")
# features_columns.remove("RestingECG")
features_columns

id = test["id"]
test = test.drop(columns = ["id"])
# test = test.drop(columns = ["RestingECG"])

features = train[features_columns]
target = train.HeartDisease
target

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(features, target, test_size=0.20, random_state=13981318)

Training

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

max_accuracy = 0

for x in range(2000):
    rf = RandomForestClassifier(random_state=x)
    rf.fit(X_train,Y_train)
    Y_pred_rf = rf.predict(X_test)
    current_accuracy = round(accuracy_score(Y_pred_rf,Y_test)*100,2)
    if(current_accuracy>max_accuracy):
        max_accuracy = current_accuracy
        best_x = x

rf = RandomForestClassifier(random_state = best_x)
rf.fit(X_train,Y_train)
Y_pred_rf = rf.predict(X_test)

accuracy = round(accuracy_score(Y_pred_rf,Y_test)*100,2)
accuracy

94.57

Testing

In [12]:
prediction = pd.DataFrame(rf.predict(test), columns = ["output"])
submission = pd.concat([id, prediction], axis = 1)
submission.to_csv("RandomForest.csv", index = False)